In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[74, 65, 72, 62, 75, 58, 65, 78],
        [57,  1, 54, 67, 57,  1, 72, 73],
        [78,  1, 54, 67, 57,  1, 55, 71],
        [61, 58, 67,  1, 54, 67,  1, 54]], device='cuda:0')
targets:
tensor([[65, 72, 62, 75, 58, 65, 78,  1],
        [ 1, 54, 67, 57,  1, 72, 73, 71],
        [ 1, 54, 67, 57,  1, 55, 71, 68],
        [58, 67,  1, 54, 67,  1, 54, 73]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Q5Ver'q]o;[Zo*")0eT&HkMzIeTAD﻿&.-ok5﻿gIfJ*s,*a)i5OXFdyX:zl07v]0mnq"&7h. CN;UnZfRLiA.'g. X,_;BEPkj9R;x(s
 qIMolv?I&MZfwt:NfPiAebJ4r-bX)Mt﻿84K_E*BBVaNV1eGx PBMTe0p A'YFV_LEZj3?IvcqWd,5[Ws)lgf1F;&Bz'K Q)[baCCsiDrr.viAm!Y5n1gLKjVTv,ZJ)U3?1rTt;QlVs9UW]5gBxI5,7Dph)YH3?OcMMYLX37Fc
q;[aLG,lElLYfcw
["67*ciA)x-6[JO2Z9nKOqkcphYanWspi;MK]zaM.zX.M4O0-n;&vbmYmR!0Q"3FbyU3ml7UBjsbq85]g
iA"o*nI9lQ.qf﻿V,RJfwYcqfWbn-P:F-saB_nv]ITt﻿sm)ifwtbEOkX.au9a,.anqd[8xi*fu[Okf!SBNSPAJ)(0ft6LEUW]y 7'
m8lL[WkiQXZ4!KQ?NSkfB]u Gk


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.666, val loss: 4.650
step: 250, train loss: 4.609, val loss: 4.585
step: 500, train loss: 4.535, val loss: 4.545
step: 750, train loss: 4.467, val loss: 4.465
4.566575050354004


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


W:E n-6567ypO2:O. w
KgIlgAwDr5,n567*v0eb(f6I5M,KCVxCH*﻿2-wZjwx3?We[.bXar2)ly[8!_,_E[LG')iJ)H-ybOph9MDv!RMop(lp6rgVaYiAb2o*_dW﻿egt6n9mSr8.o4i]MtPj9lc,ugo]HE0phrSPkh
YmSQV0fJ 4tH.u
_Qv_zy&﻿LrI]LvA,-p;ph:xs u?Usi45X.08 hOBRx,qp*mGtF7l1
LspaYzCI[Fo,bIYMgta,Dcz93Fo1Kuj.3CUdYXPBUP'O:J
Yak,V4?UBxb]M5h.Y_E.1TzR(f'paL2jf'v82ooX.rzaRI1T3?OzG(lvY0fq["vl 2;[w3XPUXkfT,6-Kwrzen1PQ"84!NtX?. QG-JQf)01VacS﻿Q5gV84j2FGKoo'x,8F,daFOVn19HKuvr,wsphglQAbL.ZKYV
-n
3[qkbq8un52H"hG-J4&-pneesinY3Fw!q2H7cALHFkX.x(O)Fy7o*oo


In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is ", context, "target is ", target)

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)


In [15]:
a = torch.rand(2,3,5)
X, Y, Z = a.shape
a = a.view(X,Y,Z)
print(a.shape)

torch.Size([2, 3, 5])
